In [2]:
import rioxarray as rxr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import os

In [3]:
#read csv file
df = pd.read_csv('stations.csv')

In [4]:
df

,Id,Name,Country,Position,PI names,Site type,Elevation above sea,Station class,Labeling date
0,http://meta.icos-cp.eu/resources/stations/ES_D...,Schechenfilz Nord,Germany (DE),11.3275 47.80639,Schmid,wetland,590.00,Associated,NaN
1,http://meta.icos-cp.eu/resources/stations/ES_F...,Hyytiala,Finland (FI),24.29477 61.84741,Mammarella,evergreen needleleaf forests,181.00,1,2018-11-30
2,http://meta.icos-cp.eu/resources/stations/ES_F...,Bilos,France (FR),-0.956092 44.493652,Domec,evergreen needleleaf forests,39.18,2,2019-11-21
3,http://meta.icos-cp.eu/resources/stations/ES_S...,Svartberget,Sweden (SE),19.7745 64.25611,Peichl,evergreen needleleaf forests,267.00,2,2019-05-22
4,http://meta.icos-cp.eu/resources/stations/AS_LMP,Lampedusa,Italy (IT),12.6322 35.5181,Di Iorio Piacentino Sferlazzo,marine remote,45.00,2,2020-05-27
...,...,...,...,...,...,...,...,...,...
173,http://meta.icos-cp.eu/resources/stations/ES_F...,Lettosuo,Finland (FI),23.95952 60.64183,Korkiakoski,evergreen needleleaf forests,125.00,Associated,2018-11-30
174,http://meta.icos-cp.eu/resources/stations/AS_HTM,Hyltemossa,Sweden (SE),13.4189 56.0976,Biermann Heliasz,tall tower,115.00,1,2018-05-31
175,http://meta.icos-cp.eu/resources/stations/OS_48MB,MIRAMARE,Italy (IT),13.708 45.698,Giani,fixed station,-2.00,2,2022-11-23
176,SE-Oes,Östergarnsholm,Sweden (SE),18.98415 57.4301,Rutgersson,water bodies,NaN,Associated,NaN


In [5]:
# Split the "Position" column into two new columns: Lon and Lat
df[['Longitude', 'Latitude']] = df['Position'].str.split(' ', expand=True)

# Convert them to float
df['Longitude'] = df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Display the first rows of the GeoDataFrame
gdf.head()

,Id,Name,Country,Position,PI names,Site type,Elevation above sea,Station class,Labeling date,Longitude,Latitude,geometry
0,http://meta.icos-cp.eu/resources/stations/ES_D...,Schechenfilz Nord,Germany (DE),11.3275 47.80639,Schmid,wetland,590.00,Associated,NaN,11.327500,47.806390,POINT (11.3275 47.80639)
1,http://meta.icos-cp.eu/resources/stations/ES_F...,Hyytiala,Finland (FI),24.29477 61.84741,Mammarella,evergreen needleleaf forests,181.00,1,2018-11-30,24.294770,61.847410,POINT (24.29477 61.84741)
2,http://meta.icos-cp.eu/resources/stations/ES_F...,Bilos,France (FR),-0.956092 44.493652,Domec,evergreen needleleaf forests,39.18,2,2019-11-21,-0.956092,44.493652,POINT (-0.95609 44.49365)
3,http://meta.icos-cp.eu/resources/stations/ES_S...,Svartberget,Sweden (SE),19.7745 64.25611,Peichl,evergreen needleleaf forests,267.00,2,2019-05-22,19.774500,64.256110,POINT (19.7745 64.25611)
4,http://meta.icos-cp.eu/resources/stations/AS_LMP,Lampedusa,Italy (IT),12.6322 35.5181,Di Iorio Piacentino Sferlazzo,marine remote,45.00,2,2020-05-27,12.632200,35.518100,POINT (12.6322 35.5181)


In [6]:
#print all different site types
site_types = gdf['Site type'].unique()

In [7]:
site_types

array(['wetland', 'evergreen needleleaf forests', 'marine remote',
       'open shrublands', 'grasslands', 'urban and built-up lands',
       'permanent wetlands', 'deciduous needleleaf forests', 'soop',
       'mountain', 'fixed station', 'water bodies', 'coastal',
       'evergreen broadleaf forests', 'non-forested island on sea',
       'croplands', 'deciduous broadleaf forests', 'closed shrublands',
       'tall tower', 'surface  land', 'savannas', 'forest', 'ground',
       '(tall) tower', 'continental', 'mixed forests',
       'coastal/continental', 'remote arctic', 'profiling station',
       'tower', 'grassland'], dtype=object)

In [8]:
#filter site_types on the word forest
forest_sites = gdf[gdf['Site type'].str.contains('forest', case=False)]

In [9]:
forest_sites

,Id,Name,Country,Position,PI names,Site type,Elevation above sea,Station class,Labeling date,Longitude,Latitude,geometry
1,http://meta.icos-cp.eu/resources/stations/ES_F...,Hyytiala,Finland (FI),24.29477 61.84741,Mammarella,evergreen needleleaf forests,181.0000,1,2018-11-30,24.294770,61.847410,POINT (24.29477 61.84741)
2,http://meta.icos-cp.eu/resources/stations/ES_F...,Bilos,France (FR),-0.956092 44.493652,Domec,evergreen needleleaf forests,39.1800,2,2019-11-21,-0.956092,44.493652,POINT (-0.95609 44.49365)
3,http://meta.icos-cp.eu/resources/stations/ES_S...,Svartberget,Sweden (SE),19.7745 64.25611,Peichl,evergreen needleleaf forests,267.0000,2,2019-05-22,19.774500,64.256110,POINT (19.7745 64.25611)
8,http://meta.icos-cp.eu/resources/stations/ES_S...,Norunda,Sweden (SE),17.479504 60.0865,Kljun,evergreen needleleaf forests,45.0000,2,2018-11-30,17.479504,60.086500,POINT (17.4795 60.0865)
10,http://meta.icos-cp.eu/resources/stations/ES_I...,Torgnon-LD,Italy (IT),7.56089 45.82376,Galvagno,deciduous needleleaf forests,2100.0000,Associated,2024-05-14,7.560890,45.823760,POINT (7.56089 45.82376)
14,http://meta.icos-cp.eu/resources/stations/ES_C...,Bily Kriz forest,Czech Republic (CZ),18.536882 49.502075,Šigut,evergreen needleleaf forests,875.0000,2,2022-05-18,18.536882,49.502075,POINT (18.53688 49.50208)
17,http://meta.icos-cp.eu/resources/stations/ES_N...,Hurdal,Norway (NO),11.07949 60.37163,Lange,evergreen needleleaf forests,275.1308,2,2023-11-15,11.079490,60.371630,POINT (11.07949 60.37163)
23,http://meta.icos-cp.eu/resources/stations/ES_F...,Sodankyla,Finland (FI),26.63859 67.36239,Aurela,evergreen needleleaf forests,180.0000,1,2023-05-23,26.638590,67.362390,POINT (26.63859 67.36239)
26,http://meta.icos-cp.eu/resources/stations/ES_G...,Guyaflux,French Guiana (GF),-52.9248 5.2787,Bonal,evergreen broadleaf forests,40.0000,Associated,2019-11-21,-52.924800,5.278700,POINT (-52.9248 5.2787)
27,http://meta.icos-cp.eu/resources/stations/ES_I...,Renon,Italy (IT),11.43369 46.58686,Montagnani,evergreen needleleaf forests,1735.6000,2,2021-11-17,11.433690,46.586860,POINT (11.43369 46.58686)


In [10]:
import leafmap


In [39]:
from typing import Union, Optional
import ipyleaflet
import pandas as pd
import os


In [65]:
def add_labels(
    self,
    data: Union[str, pd.DataFrame],
    column: str,
    font_size: Optional[str] = "12pt",  # Increase this if needed
    font_color: Optional[str] = "black",
    font_family: Optional[str] = "arial",
    font_weight: Optional[str] = "normal",
    x: Optional[str] = "longitude",
    y: Optional[str] = "latitude",
    draggable: Optional[bool] = True,
    x_offset: int = 5,  # Horizontal offset
    y_offset: int = -2,  # Vertical offset
    background_color: Optional[str] = None,  # Background color (None = transparent)
    border_radius: Optional[str] = "4px",  # Rounded corners
    layer_name: Optional[str] = "Labels",
    **kwargs,
):
    """Adds a label layer to the map with dynamically sized background."""

    import warnings
    warnings.filterwarnings("ignore")

    if isinstance(data, pd.DataFrame):
        df = data
        if "geometry" in data.columns or ("geom" in data.columns):
            df[x] = df.centroid.x
            df[y] = df.centroid.y

    elif isinstance(data, str):
        ext = os.path.splitext(data)[1]
        if ext == ".csv":
            df = pd.read_csv(data)
        elif ext in [".geojson", ".json", ".shp", ".gpkg"]:
            try:
                import geopandas as gpd
                df = gpd.read_file(data)
                df[x] = df.centroid.x
                df[y] = df.centroid.y
            except Exception:
                print("geopandas is required to read geojson.")
                return
    else:
        raise ValueError(
            "data must be a pd.DataFrame, gpd.GeoDataFrame, or an ee.FeatureCollection."
        )

    if column not in df.columns:
        raise ValueError(f"column must be one of {', '.join(df.columns)}.")
    if x not in df.columns:
        raise ValueError(f"x must be one of {', '.join(df.columns)}.")
    if y not in df.columns:
        raise ValueError(f"y must be one of {', '.join(df.columns)}.")

    try:
        size = int(font_size.replace("pt", ""))
    except:
        raise ValueError("font_size must be something like '10pt'")

    # Background style with auto-sizing
    bg_style = (
        f"background-color: {background_color}; border-radius: {border_radius}; "
        f"display: inline-block; width: max-content; padding: 1px 4px; line-height: 1;"
        if background_color else ""
    )

    labels = []
    for index in df.index:
        html = f'''
        <div style="font-size: {font_size}; color: {font_color}; font-family: {font_family}; 
                    font-weight: {font_weight}; {bg_style}">
            {df[column][index]}
        </div>'''

        marker = ipyleaflet.Marker(
            location=[df[y][index], df[x][index]],
            icon=ipyleaflet.DivIcon(
                icon_size=(1, 1),
                icon_anchor=(size + x_offset, size + y_offset),  # Offset applied here
                html=html,
                **kwargs,
            ),
            draggable=draggable,
        )
        labels.append(marker)

    layer_group = ipyleaflet.LayerGroup(layers=labels, name=layer_name)
    self.add(layer_group)
    self.labels = layer_group




In [69]:
# Override leafmap's existing add_labels function with our custom one
import leafmap

leafmap.Map.add_labels = add_labels  # Replace the method with our custom version

# Create the map centered on Europe
m = leafmap.Map(center=(50, 10), zoom=4)
m.add_basemap('Esri.WorldImagery')

# Add markers for the forest sites
m.add_markers(
    forest_sites,
    x='Longitude',
    y='Latitude',
    popup='Name',
    shape='circle',
    radius=3,
    color='#14C9AC',
    fill_opacity=1
)

# Now use the updated add_labels function
m.add_labels(
    forest_sites,
    column="Name",
    font_size="9pt",  # Adjust font size if needed
    font_family="gill sans mt",
    font_color="black",
    x_offset=-5,  # Move labels to the right
    y_offset=-8,  # Adjust vertical position
    background_color="rgba(255, 255, 255, 0.5)",  # Semi-transparent white background
    border_radius="6px",  # Slightly rounded corners
)



m  # Display the updated map


Map(center=[50, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [68]:
m.save('forest_sites.html')